In [ ]:
import load_dataset_1week
dataset = load_dataset_1week.load_dataset_minute(5161)

In [ ]:
feature = dataset.drop(columns=['datetime', 'internet'])
target = dataset[['datetime', 'internet']]

In [ ]:
import split_data
train_x, test_x, train_y, test_y, y_datetime = split_data.split_data_8_2(feature, target)

In [ ]:
import normalize_train_test
train_x, test_x, train_y, test_y, mms_y = normalize_train_test.apply_MMS(train_x, test_x, train_y, test_y)

In [ ]:
import set_timesteps
timesteps = 36
x_train, x_test, y_train, y_test = set_timesteps.create_timeseries_data(train_x, test_x, train_y, test_y, timesteps)

In [ ]:
import build_model
time_unit = 'minute'
model, history = build_model.build_model_GRU_Optuna(x_train, y_train, timesteps, time_unit)

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot') #グラフスタイル

plt.figure(figsize=(15, 8))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Valid Loss')
plt.title('Model Loss (GRU)')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(loc='upper right')

# y軸の上限値を指定（例：0から1.0までの範囲に設定）
plt.ylim(0, 0.1)
plt.show()

In [ ]:
# テストデータに対する予測を行い、評価結果を表示
import evaluate_model_performance
y_test_pred, y_test_true = evaluate_model_performance.evaluate_model(x_test, y_test, mms_y, model)

In [ ]:
# テストデータ(青)と予測(黄色)を描画
datetime = y_datetime[timesteps:]

plt.figure(figsize=(30,8))
plt.plot(datetime, y_test_true, linestyle='--', color='blue', label='true')
plt.plot(datetime, y_test_pred, color='orange', label='pred')
plt.ylabel('value')
plt.xlabel('datetime')
plt.legend(loc='upper right')
plt.show()